####To Do's:
1)Clean email body
* Need to remove proper nouns
* Need to remove any text after "U.S. Department of State","Sent from Verizon"
* Remove email addresses 
* Remove dates especially of the format "Monday, January 18, 2010 11:52 AM"
* Remove times
* Remove web addresses
* Need to remove any strange non-English words
* How do I deal with emails that list schedule for the day?
* Change all occurences of "pis" to "pls". Assuming OCR error.

2)Run K-Means Clustering
3)Run LDA 

* After defining topics , determine similarity between emails for clustering. Possibly clustering texts with KMeans
* Need to stem words? Keep a word dictionary for all stem words, they don't make sense otherwise
* Look specifically at emails with **fwd** 

* Locality-sensitive hashing
* Look into lemmatization
* Chunking 

###Import Libraries

In [7]:
import pandas as pd
import numpy as np
import sqlite3
import itertools
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
% matplotlib inline
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models, similarities, matutils
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

In [8]:
con = sqlite3.connect('input/database.sqlite')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
[x for x in cursor.fetchall()]

[(u'Emails',), (u'Persons',), (u'Aliases',), (u'EmailReceivers',)]

In [9]:
emails= pd.read_sql_query("Select * From Emails where ExtractedBodyText!= ''",con)
persons=pd.read_sql_query("Select * From Persons",con)
longemails= pd.read_sql_query("Select * From Emails where length(ExtractedBodyText)>500 \
                                and ExtractedBodyText!= ''",con)
aliases=pd.read_sql_query("Select * From Aliases", con)

In [10]:
print "Number of Emails: %d" % emails.shape[0]

Number of Emails: 6742


In [11]:
import random
my_randoms = random.sample(xrange(6742), 10)
count=0
for i in my_randoms:
    count+=1
    print ()
    print ("Random Email #%d" %count)
    print ()
    print(emails["ExtractedBodyText"][i])
    print ()

()
Random Email #1
()
Mills, Cheryl D <MillsCD@state.gov>
Wednesday, December 22, 2010 11:48 AM
FW: QDDR FOLLOWUP
Sigh — despite my having advised otherwise ...
()
()
Random Email #2
()
Yep! Received 13. Printed and on your desk.
>
()
()
Random Email #3
()
One thing I missed:
Currency deferral decision likely tonight.
()
()
Random Email #4
()
Can you send me the language we prefer in a resolution?
()
()
Random Email #5
()
For hRC's review it's time sensitive and due today, so if you can print for her that would be ideal.
'anks.
ca.).
()
()
Random Email #6
()
Oh, dear. I guess I'd rather leave at 2 but does that present problems I don't know about?
()
()
Random Email #7
()
Importance: High
FYI
- - —
()
()
Random Email #8
()
Ok to set up call. What's his first name?
()
()
Random Email #9
()
Pat and I are looking at this will check in am B5
()
()
Random Email #10
()
FYI — Bill Burns cannot move his meeting.
()


###Clean up text of email body

In [12]:
# Uncomment the next line to download stop words if it's not already installed.
#nltk.download()

In [13]:
#Define email_to_words function to clean email body
#import email_to_words as email_to_words
%run ./email_to_words2.py

In [14]:
# Get the number of emails based on the dataframe column size
num_emails = emails["ExtractedBodyText"].size

# Initialize an empty list to hold the clean reviews
clean_emails = []
stemmed_emails = []
# Loop over each email; create an index i that goes from 0 to the length
# of the emails 
for i in xrange( 0, num_emails ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print ("Cleaning email %d of %d\n" % ( i+1, num_emails ))                                                             
  
    # Call our function for each one, and add the result to the list of
    # clean reviews
    try:
        clean_emails.append( email_to_words( emails["ExtractedBodyText"][i] , english=False) )

    except Exception as e:
        clean_emails.append( email_to_words("I'm a placeholder sentence."), english=False)
        print ("Execption raised:", e)

Cleaning email 1000 of 6742

Cleaning email 2000 of 6742

Cleaning email 3000 of 6742

Cleaning email 4000 of 6742

Cleaning email 5000 of 6742

Cleaning email 6000 of 6742



In [15]:
#Remove any empty emails
clean_emails=[w for w in clean_emails if len(w)>0]

In [16]:
len(clean_emails)

6339

In [29]:
import pickle  # or import cPickle as pickle
 
# Create dictionary, list, etc. 
# Write to file
pickle.dump( clean_emails, open( "output/clean_emails.p", "wb" ) )


####Tokenize and Stem 

In [17]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [p_stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [18]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in clean_emails:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'cleaned_emails', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [19]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print ('There are ' + str(vocab_frame.shape[0]) + ' items/words in vocab_frame')

There are 292327 items/words in vocab_frame


####TF-IDF

In [454]:
#Define vectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', use_idf=True, tokenizer=tokenize_and_stem)

#Fit the vectorizer
%time tfidf_matrix=tfidf_vectorizer.fit_transform(clean_emails)

CPU times: user 11 s, sys: 147 ms, total: 11.2 s
Wall time: 11.2 s


In [455]:
print(tfidf_matrix.shape)

(6340, 17201)


In [456]:
terms=tfidf_vectorizer.get_feature_names()


### K-Means Clustering

In [457]:
numb_clusters=10

kmeans=KMeans(n_clusters=numb_clusters, random_state=20151106)

%time kmeans.fit(tfidf_matrix)

clusters = kmeans.labels_.tolist()

CPU times: user 5.26 s, sys: 46.9 ms, total: 5.3 s
Wall time: 5.42 s


In [458]:
emails={'emails':clean_emails, 'cluster':clusters}

In [459]:
cluster_frame=pd.DataFrame(emails, index=[clusters], columns=['cluster'])

In [460]:
#Number of emails per cluster
cluster_frame['cluster'].value_counts()

5    3663
6    1308
2     468
7     167
8     167
1     161
4     146
9     117
0      77
3      66
Name: cluster, dtype: int64

In [461]:
from __future__ import print_function

print("Top terms pers cluster:")
print ()
#Sort cluster centers by proximity to centroid
order_centroids = kmeans.cluster_centers_.argsort()[:,::-1]

for i in range(numb_clusters):
    print("Cluster %d terms:" % i, end='')
    
    for ind in order_centroids[i,:6]:
        
        print (" %s" % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=",")
    print()
    print()

Top terms pers cluster:

Cluster 0 terms: release, fyi, declassify, reason, fedex, jake,

Cluster 1 terms: thank, please, help, know, let, great,

Cluster 2 terms: fyi, article, minute, stories, process, post,

Cluster 3 terms: oscar, asap, print, flores, sheet, statement,

Cluster 4 terms: print, please, pis, copied, thank, haiti,

Cluster 5 terms: tomorrow, fyi, please, know, want, today,

Cluster 6 terms: talk, work, good, think, time, want,

Cluster 7 terms: offical, depart, state, valmoro, assist, room,

Cluster 8 terms: email, please, thank, travel, access, assist,

Cluster 9 terms: yes, work, thank, set, come, sorry,



### Latent Dirichlet Allocation

In [20]:
from gensim import corpora, models, similarities 
#tokenize
%time token_emails = [tokenize_and_stem(text) for text in clean_emails]

CPU times: user 10.4 s, sys: 110 ms, total: 10.5 s
Wall time: 10.7 s


In [21]:
# turn our tokenized documents into a id <-> term dictionary
%time dictionary = corpora.Dictionary(token_emails)

#remove extremes
%time dictionary.filter_extremes(no_below=1, no_above=0.8)

dictionary.compactify()

# convert tokenized documents into a document-term matrix
%time corpus = [dictionary.doc2bow(text) for text in token_emails]


CPU times: user 434 ms, sys: 8.75 ms, total: 443 ms
Wall time: 441 ms
CPU times: user 35.4 ms, sys: 922 µs, total: 36.3 ms
Wall time: 36.4 ms
CPU times: user 339 ms, sys: 6.48 ms, total: 345 ms
Wall time: 345 ms


In [557]:
topics_range=(4,6,8,10,12)
for i in topics_range:
    np.random.seed(20111106)
    # generate LDA model
    %time ldamodel = models.ldamodel.LdaModel(corpus, num_topics=i, id2word = dictionary, passes=100, alpha='auto')
    ldamodel.save('output/final_topic%d.model'%i)


CPU times: user 6min 9s, sys: 2.49 s, total: 6min 11s
Wall time: 6min 16s
CPU times: user 6min 12s, sys: 4.85 s, total: 6min 16s
Wall time: 6min 26s
CPU times: user 6min 4s, sys: 4.1 s, total: 6min 8s
Wall time: 6min 13s
CPU times: user 5min 59s, sys: 3.6 s, total: 6min 3s
Wall time: 6min 7s
CPU times: user 6min 3s, sys: 4.97 s, total: 6min 8s
Wall time: 6min 13s


In [30]:
for i in topics_range:
    final=models.ldamodel.LdaModel.load('output/final_topic%d.model'%i)
    count=0
    print ("%s TOPICS"%i)
    for i in (final.print_topics(num_words=8)):
        count+=1
        print ()
        print ('Topic# %s :' %count, i)
        print ()

NameError: name 'topics_range' is not defined

In [26]:
final=models.ldamodel.LdaModel.load('output/final_topic10.model')


###Evaluation

In [4]:
# select top 10 words for each of the 20 LDA topics
top_words = [[word for _, word in final.show_topic(topicno, topn=8)] for topicno in range(final.num_topics)]
print(top_words)

[[u'offic', u'depart', u'meet', u'room', u'state', u'arriv', u'rout', u'confer'], [u'call', u'see', u'get', u'want', u'know', u'work', u'thank', u'talk'], [u'pleas', u'print', u'messag', u'list', u'thank', u'qddr', u'copi', u'add'], [u'releas', u'part', u'state', u'new', u'team', u'chapter', u'valmoro', u'branch'], [u'american', u'state', u'new', u'presid', u'one', u'year', u'said', u'obama'], [u'hous', u'obama', u'white', u'bill', u'said', u'presid', u'staff', u'senat'], [u'fyi', u'israel', u'isra', u'parti', u'palestinian', u'peac', u'arab', u'negoti'], [u'work', u'develop', u'state', u'new', u'polici', u'peopl', u'issu', u'world'], [u'republican', u'democrat', u'vote', u'senat', u'percent', u'elect', u'parti', u'candid'], [u'koch', u'tea', u'right', u'beck', u'parti', u'movement', u'book', u'skousen']]


In [5]:
# get all top 50 words in all 20 topics, as one large set
all_words = set(itertools.chain.from_iterable(top_words))

print("Can you find the word intruder?")

# for each topic, replace a word at a different index, to make it more interesting
replace_index = np.random.randint(0, 8, final.num_topics)

replacements = []
for topicno, words in enumerate(top_words):
    other_words = all_words.difference(words)
    replacement = np.random.choice(list(other_words))
    replacements.append((words[replace_index[topicno]], replacement))
    words[replace_index[topicno]] = replacement
    print("%i: %s" % (topicno, ' '.join(words[:10])))

Can you find the word intruder?
0: offic depart meet room state arriv list confer
1: call see get want know work develop talk
2: pleas print messag list thank year copi add
3: releas part arriv new team chapter valmoro branch
4: american state new rout one year said obama
5: talk obama white bill said presid staff senat
6: fyi israel isra parti palestinian peac arab hous
7: work develop year new polici peopl issu world
8: republican democrat vote senat percent fyi parti candid
9: koch hous right beck parti movement book skousen


In [575]:
print("Actual replacements were:")
print(list(enumerate(replacements)))

Actual replacements were:
[(0, (u'depart', u'republican')), (1, (u'work', u'valmoro')), (2, (u'add', u'candid')), (3, (u'part', u'negoti')), (4, (u'american', u'messag')), (5, (u'senat', u'print')), (6, (u'arab', u'messag')), (7, (u'state', u'right')), (8, (u'republican', u'messag')), (9, (u'parti', u'confer'))]


In [559]:
def intra_inter(model, test_docs, num_pairs=10000):
    # split each test document into two halves and compute topics for each half
    part1 = [model[dictionary.doc2bow(tokens[: len(tokens) / 2])] for tokens in test_docs]
    part2 = [model[dictionary.doc2bow(tokens[len(tokens) / 2 :])] for tokens in test_docs]
    
    # print computed similarities (uses cossim)
    print("average cosine similarity between corresponding parts (higher is better):")
    print(np.mean([matutils.cossim(p1, p2) for p1, p2 in zip(part1, part2)]))

    random_pairs = np.random.randint(0, len(test_docs), size=(num_pairs, 2))
    print("average cosine similarity between random parts (lower is better):")    
    print(np.mean([matutils.cossim(part1[i[0]], part2[i[1]]) for i in random_pairs]))

In [560]:
for i in topics_range:
    final=models.ldamodel.LdaModel.load('final_topic%d.model'%i)
    print("LDA results %s TOPICS:"%i)
    intra_inter(final, test_docs=token_emails)

LDA results 4 TOPICS:
average cosine similarity between corresponding parts (higher is better):
0.861707880884
average cosine similarity between random parts (lower is better):
0.691270549601
LDA results 6 TOPICS:
average cosine similarity between corresponding parts (higher is better):
0.839454637874
average cosine similarity between random parts (lower is better):
0.672448411281
LDA results 8 TOPICS:
average cosine similarity between corresponding parts (higher is better):
0.833823493316
average cosine similarity between random parts (lower is better):
0.648171243314
LDA results 10 TOPICS:
average cosine similarity between corresponding parts (higher is better):
0.832964800841
average cosine similarity between random parts (lower is better):
0.659063567498
LDA results 12 TOPICS:
average cosine similarity between corresponding parts (higher is better):
0.839291530635
average cosine similarity between random parts (lower is better):
0.666771757282


###Results


In [27]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [526]:
gensimvis.prepare?


In [28]:
vis_data = gensimvis.prepare(final, corpus, dictionary)
pyLDAvis.display(vis_data)

/Users/banti/Applications/anaconda/lib/python2.7/site-packages/pyLDAvis/_prepare.py:283: FutureWarning: order is deprecated, use sort_values(...)
  topic_proportion = (topic_freq / topic_freq.sum()).order(ascending=False)
/Users/banti/Applications/anaconda/lib/python2.7/site-packages/pyLDAvis/_prepare.py:154: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  sort('saliency', ascending=False). \
/Users/banti/Applications/anaconda/lib/python2.7/site-packages/pyLDAvis/_prepare.py:203: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  return token_table.sort(['Term', 'Topic'])
/Users/banti/Applications/anaconda/lib/python2.7/site-packages/pyLDAvis/_prepare.py:132: FutureWarning: order is deprecated, use sort_values(...)
  return relevance.T.apply(lambda s: s.order(ascending=False).index).head(R)
/Users/banti/Applications/anaconda/lib/python2.7/site-packages/pyLDAvis/_prepare.py:132: FutureWarning: order is deprecated, use sort_values(..